# News Sentiment Infomation

Display sentimental polarity pie chart and trend chart 

# Load Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')

In [3]:
df.head(1)

,item_id,title,category,content,link,date,photo_link,tokens_v2,top_keys_freq,summary,sentiment
0,aipl_20220124_1001,殲16D首擾台 專家示警：國軍應強化電子作戰,政治,中共殲16D新型電戰機今天首次擾台，學者及退將分析，殲16D可能已量產並進行實戰化運用，國軍...,https://www.cna.com.tw/news/aipl/202201240357....,2022-01-24,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,"['中共', '新型', '電戰機', '學者', '退將', '分析', '量產', '進...","[('中共', 7), ('台灣', 7), ('電子', 6), ('美軍', 6), (...","['其中殲16D新型電戰機為首次現蹤', '中共殲16D新型電戰機今天首次擾台', '13架...",0.96


In [4]:
df.shape

(26179, 11)

In [5]:
df.date.max()

'2022-04-23'

In [6]:
df.date.min()

'2022-01-24'

# Step 1: Filter news articles using the following function

In [7]:
from datetime import datetime, timedelta

In [8]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
# It is the same as filter_dataFrame_fullText()此函數名稱之前是用filter_dataFrame_fullText()
def filter_df_via_content(query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


# Step 2: Sentimental polarity score計算整體情緒分數(影響力)

Count number of article with Positive, Negative, and Neutral polarity (objective polarity)

0~1  

In [9]:
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0} # count次數
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0} # percentage百分比
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.6: #大於0.6為正向 
            sentiCount['pos'] += 1
        elif senti <= 0.4: #小於0.4為負向
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1 #介於中間為中立
    for polar in sentiCount :
        try:
            sentiPercnt[polar]=int(sentiCount[polar]/numberOfArticle*100)
            # sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle,2) # 0.75
        except:
            sentiPercnt[polar] = 0 # 若分母 numberOfArticle=0會報錯
    return sentiCount, sentiPercnt


In [10]:
query_keywords = ['黃珊珊']
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_content(query_keywords, cond, cate, weeks)
sentiCount, sentiPercnt = get_article_sentiment(df_query)

In [11]:
sentiCount

{'pos': 76, 'neg': 34, 'obj': 23}

In [15]:
sentiPercnt

{'pos': 57, 'neg': 25, 'obj': 17}

In [12]:
'正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),str(sentiPercnt['neg']))

'正向:57%, 中立:17%, 負向:25%'

In [13]:
[sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

[57, 17, 25]

# Step 3: Count positive and negative articles and get line chart data

The number of articles for positive and negative sentimental polarity.

This is the data for drawing daily-basis line chart on homepage.

    The data format for the line chart is as follows.

    [{'x': '2020-03-05', 'y': 1},
    {'x': '2020-03-06', 'y': 0},
    {'x': '2020-03-07', 'y': 0},
    {'x': '2020-03-08', 'y': 0},
    {'x': '2020-03-09', 'y': 0},
    {'x': '2020-03-10', 'y': 5},
    {'x': '2020-03-11', 'y': 5},
    {'x': '2020-03-12', 'y': 7},
    {'x': '2020-03-13', 'y': 62},
    {'x': '2020-03-14', 'y': 29}]

In [14]:
def get_daily_basis_sentiment_count(df_query, sentiment_type='pos', freq_type='D'):

    # 自訂正負向中立的標準，sentiment score是機率值，介於0~1之間
    # Using lambda to determine if an article is postive or not.
    if sentiment_type == 'pos':
        lambda_function = lambda senti: 1 if senti >= 0.6 else 0 #大於0.6為正向 
    elif sentiment_type == 'neg':
        lambda_function = lambda senti: 1 if senti <= 0.4 else 0 #小於0.4為負向
    elif sentiment_type == 'neutral':
        lambda_function = lambda senti: 1 if senti > 0.4 & senti < 0.4 else 0 #介於中間為中立
    else:
        return None 
        
    freq_df = pd.DataFrame({'date_index': pd.to_datetime(df_query.date),
                             'frequency': [lambda_function(senti) for senti in df_query.sentiment]})
    # Group rows by the date to the daily number of articles. 加總合併同一天新聞，篇數就被計算好了
    freq_df_group = freq_df.groupby(pd.Grouper(key='date_index', freq=freq_type)).sum()
    
    # 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
    freq_df_group = freq_df_group.reset_index()
    #freq_df_group.reset_index(inplace=True)
    
    # x,y，用於畫趨勢線圖
    xy_line_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_df_group.date_index,freq_df_group.frequency)]
    return xy_line_data

In [15]:
query_keywords = ['黃珊珊']
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_content(query_keywords, cond, cate, weeks)

In [16]:
df_query.shape

(133, 11)

In [17]:
get_daily_basis_sentiment_count(df_query, sentiment_type='pos')

[{'x': '2022-01-29', 'y': 0},
 {'x': '2022-01-30', 'y': 0},
 {'x': '2022-01-31', 'y': 0},
 {'x': '2022-02-01', 'y': 0},
 {'x': '2022-02-02', 'y': 1},
 {'x': '2022-02-03', 'y': 1},
 {'x': '2022-02-04', 'y': 0},
 {'x': '2022-02-05', 'y': 3},
 {'x': '2022-02-06', 'y': 2},
 {'x': '2022-02-07', 'y': 1},
 {'x': '2022-02-08', 'y': 1},
 {'x': '2022-02-09', 'y': 1},
 {'x': '2022-02-10', 'y': 1},
 {'x': '2022-02-11', 'y': 1},
 {'x': '2022-02-12', 'y': 0},
 {'x': '2022-02-13', 'y': 0},
 {'x': '2022-02-14', 'y': 2},
 {'x': '2022-02-15', 'y': 0},
 {'x': '2022-02-16', 'y': 0},
 {'x': '2022-02-17', 'y': 2},
 {'x': '2022-02-18', 'y': 3},
 {'x': '2022-02-19', 'y': 0},
 {'x': '2022-02-20', 'y': 0},
 {'x': '2022-02-21', 'y': 0},
 {'x': '2022-02-22', 'y': 0},
 {'x': '2022-02-23', 'y': 0},
 {'x': '2022-02-24', 'y': 0},
 {'x': '2022-02-25', 'y': 1},
 {'x': '2022-02-26', 'y': 0},
 {'x': '2022-02-27', 'y': 0},
 {'x': '2022-02-28', 'y': 0},
 {'x': '2022-03-01', 'y': 0},
 {'x': '2022-03-02', 'y': 0},
 {'x': '20

In [18]:
get_daily_basis_sentiment_count(df_query, sentiment_type='neg')

[{'x': '2022-01-29', 'y': 0},
 {'x': '2022-01-30', 'y': 0},
 {'x': '2022-01-31', 'y': 0},
 {'x': '2022-02-01', 'y': 0},
 {'x': '2022-02-02', 'y': 1},
 {'x': '2022-02-03', 'y': 0},
 {'x': '2022-02-04', 'y': 0},
 {'x': '2022-02-05', 'y': 0},
 {'x': '2022-02-06', 'y': 1},
 {'x': '2022-02-07', 'y': 1},
 {'x': '2022-02-08', 'y': 1},
 {'x': '2022-02-09', 'y': 0},
 {'x': '2022-02-10', 'y': 1},
 {'x': '2022-02-11', 'y': 0},
 {'x': '2022-02-12', 'y': 2},
 {'x': '2022-02-13', 'y': 0},
 {'x': '2022-02-14', 'y': 1},
 {'x': '2022-02-15', 'y': 0},
 {'x': '2022-02-16', 'y': 1},
 {'x': '2022-02-17', 'y': 1},
 {'x': '2022-02-18', 'y': 0},
 {'x': '2022-02-19', 'y': 0},
 {'x': '2022-02-20', 'y': 0},
 {'x': '2022-02-21', 'y': 1},
 {'x': '2022-02-22', 'y': 1},
 {'x': '2022-02-23', 'y': 0},
 {'x': '2022-02-24', 'y': 1},
 {'x': '2022-02-25', 'y': 0},
 {'x': '2022-02-26', 'y': 0},
 {'x': '2022-02-27', 'y': 0},
 {'x': '2022-02-28', 'y': 0},
 {'x': '2022-03-01', 'y': 0},
 {'x': '2022-03-02', 'y': 0},
 {'x': '20

## How does this work?

In [22]:
# This is what we used to calculate daily frequency of keyword in our previous app
pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'frequency':[1 for _ in range(len(df_query))]})

,date_index,frequency
1822,2022-01-29,1
2395,2022-02-02,1
2398,2022-02-02,1
2580,2022-02-03,1
2740,2022-02-04,1
...,...,...
25482,2022-04-21,1
25519,2022-04-21,1
25528,2022-04-21,1
25848,2022-04-22,1


In [23]:
# Now we need to modify the above line as follows.
# Using lambda to determine if an article is postive or not.
pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'frequency':[ (lambda senti: 1 if senti >= 0.6 else 0)(senti) for senti in df_query.sentiment]})

,date_index,frequency
1822,2022-01-29,0
2395,2022-02-02,0
2398,2022-02-02,1
2580,2022-02-03,1
2740,2022-02-04,0
...,...,...
25482,2022-04-21,1
25519,2022-04-21,1
25528,2022-04-21,0
25848,2022-04-22,1


In [24]:
# What is the following lambda function? It is a little bite hard to understand.
[ (lambda senti: 1 if senti >= 0.6 else 0)(senti) for senti in df_query.sentiment  ]

[0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0]

In [25]:
lambda x: 1 if x >= 0.6 else 0

<function __main__.<lambda>(x)>

In [26]:
(lambda x: 1 if x >= 0.6 else 0)(0.9)  #  f(0.9)   f: lambda x: 1 if x >= 0.6  def f:

1

In [27]:
(lambda x: 1 if x >= 0.6 else 0)(0.2)

0

In [28]:
pd.to_datetime( df_query.date )

1822    2022-01-29
2395    2022-02-02
2398    2022-02-02
2580    2022-02-03
2740    2022-02-04
           ...    
25482   2022-04-21
25519   2022-04-21
25528   2022-04-21
25848   2022-04-22
26058   2022-04-23
Name: date, Length: 133, dtype: datetime64[ns]

In [29]:
{'date_index':pd.to_datetime( df_query.date ),'frequency':[ (lambda senti: 1 if senti >= 0.7 else 0)(senti) for senti in df_query.sentiment]}

{'date_index': 1822    2022-01-29
 2395    2022-02-02
 2398    2022-02-02
 2580    2022-02-03
 2740    2022-02-04
            ...    
 25482   2022-04-21
 25519   2022-04-21
 25528   2022-04-21
 25848   2022-04-22
 26058   2022-04-23
 Name: date, Length: 133, dtype: datetime64[ns],
 'frequency': [0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0]}

In [30]:
pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'frequency':[ (lambda senti: 1 if senti >= 0.7 else 0)(senti) for senti in df_query.sentiment]})

,date_index,frequency
1822,2022-01-29,0
2395,2022-02-02,0
2398,2022-02-02,1
2580,2022-02-03,1
2740,2022-02-04,0
...,...,...
25482,2022-04-21,1
25519,2022-04-21,1
25528,2022-04-21,0
25848,2022-04-22,1


In [28]:
# a dict with 'date_index' key and 'frequency' value ==> A dict is consist of several (key, value) pairs.
{'date_index':pd.to_datetime( df_query.date ),'frequency':[ (lambda senti: 1 if senti >= 0.6 else 0)(senti) for senti in df_query.sentiment]}

{'date_index': 1822    2022-01-29
 2395    2022-02-02
 2398    2022-02-02
 2580    2022-02-03
 2740    2022-02-04
            ...    
 25482   2022-04-21
 25519   2022-04-21
 25528   2022-04-21
 25848   2022-04-22
 26058   2022-04-23
 Name: date, Length: 133, dtype: datetime64[ns],
 'frequency': [0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0]}

In [31]:
# Convert the above dict into DataFrame and assign it to query_freq
freq_df = pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'frequency':[ (lambda senti: 1 if senti >= 0.6 else 0)(senti) for senti in df_query.sentiment]})

In [32]:
freq_df

,date_index,frequency
1822,2022-01-29,0
2395,2022-02-02,0
2398,2022-02-02,1
2580,2022-02-03,1
2740,2022-02-04,0
...,...,...
25482,2022-04-21,1
25519,2022-04-21,1
25528,2022-04-21,0
25848,2022-04-22,1


In [33]:
# Group rows by the date to the daily number of articles. 加總合併同一天新聞，篇數就被計算好了
freq_df_group = freq_df.groupby(pd.Grouper(key='date_index',freq='D')).sum()

In [34]:
freq_df_group

,frequency
date_index,
2022-01-29,0
2022-01-30,0
2022-01-31,0
2022-02-01,0
2022-02-02,1
...,...
2022-04-19,1
2022-04-20,0
2022-04-21,2


In [35]:
# 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
freq_df_group.reset_index(inplace=True)
#freq_df_group = freq_df_group.reset_index() # 這樣也可以得到同樣結果


In [36]:
freq_df_group

,date_index,frequency
0,2022-01-29,0
1,2022-01-30,0
2,2022-01-31,0
3,2022-02-01,0
4,2022-02-02,1
...,...,...
80,2022-04-19,1
81,2022-04-20,0
82,2022-04-21,2
83,2022-04-22,1


In [38]:
list(zip(freq_df_group.date_index,freq_df_group.frequency))

[(Timestamp('2022-01-29 00:00:00'), 0),
 (Timestamp('2022-01-30 00:00:00'), 0),
 (Timestamp('2022-01-31 00:00:00'), 0),
 (Timestamp('2022-02-01 00:00:00'), 0),
 (Timestamp('2022-02-02 00:00:00'), 1),
 (Timestamp('2022-02-03 00:00:00'), 1),
 (Timestamp('2022-02-04 00:00:00'), 0),
 (Timestamp('2022-02-05 00:00:00'), 3),
 (Timestamp('2022-02-06 00:00:00'), 2),
 (Timestamp('2022-02-07 00:00:00'), 1),
 (Timestamp('2022-02-08 00:00:00'), 1),
 (Timestamp('2022-02-09 00:00:00'), 1),
 (Timestamp('2022-02-10 00:00:00'), 1),
 (Timestamp('2022-02-11 00:00:00'), 1),
 (Timestamp('2022-02-12 00:00:00'), 0),
 (Timestamp('2022-02-13 00:00:00'), 0),
 (Timestamp('2022-02-14 00:00:00'), 2),
 (Timestamp('2022-02-15 00:00:00'), 0),
 (Timestamp('2022-02-16 00:00:00'), 0),
 (Timestamp('2022-02-17 00:00:00'), 2),
 (Timestamp('2022-02-18 00:00:00'), 3),
 (Timestamp('2022-02-19 00:00:00'), 0),
 (Timestamp('2022-02-20 00:00:00'), 0),
 (Timestamp('2022-02-21 00:00:00'), 0),
 (Timestamp('2022-02-22 00:00:00'), 0),


In [39]:

# 有時間變數x,y，用於畫趨勢線圖
line_xy_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_df_group.date_index,freq_df_group.frequency)]

In [40]:
line_xy_data

[{'x': '2022-01-29', 'y': 0},
 {'x': '2022-01-30', 'y': 0},
 {'x': '2022-01-31', 'y': 0},
 {'x': '2022-02-01', 'y': 0},
 {'x': '2022-02-02', 'y': 1},
 {'x': '2022-02-03', 'y': 1},
 {'x': '2022-02-04', 'y': 0},
 {'x': '2022-02-05', 'y': 3},
 {'x': '2022-02-06', 'y': 2},
 {'x': '2022-02-07', 'y': 1},
 {'x': '2022-02-08', 'y': 1},
 {'x': '2022-02-09', 'y': 1},
 {'x': '2022-02-10', 'y': 1},
 {'x': '2022-02-11', 'y': 1},
 {'x': '2022-02-12', 'y': 0},
 {'x': '2022-02-13', 'y': 0},
 {'x': '2022-02-14', 'y': 2},
 {'x': '2022-02-15', 'y': 0},
 {'x': '2022-02-16', 'y': 0},
 {'x': '2022-02-17', 'y': 2},
 {'x': '2022-02-18', 'y': 3},
 {'x': '2022-02-19', 'y': 0},
 {'x': '2022-02-20', 'y': 0},
 {'x': '2022-02-21', 'y': 0},
 {'x': '2022-02-22', 'y': 0},
 {'x': '2022-02-23', 'y': 0},
 {'x': '2022-02-24', 'y': 0},
 {'x': '2022-02-25', 'y': 1},
 {'x': '2022-02-26', 'y': 0},
 {'x': '2022-02-27', 'y': 0},
 {'x': '2022-02-28', 'y': 0},
 {'x': '2022-03-01', 'y': 0},
 {'x': '2022-03-02', 'y': 0},
 {'x': '20

# Step 4: Prepare the response data 

In [39]:
def prepare_for_response(query_keywords, cond, cate, weeks):

    # Proceed filtering
    df_query = filter_df_via_content(query_keywords, cond, cate, weeks)
    
    sentiCount, sentiPercnt = get_article_sentiment(df_query)

    if weeks <= 4:
        freq_type = 'D'
    else:
        freq_type = 'W' # weekly basis

    line_data_pos = get_daily_basis_sentiment_count(df_query, sentiment_type='pos', freq_type=freq_type)
    line_data_neg = get_daily_basis_sentiment_count(df_query, sentiment_type='neg', freq_type=freq_type)

    response = {
        'sentiCount': sentiCount,
        'data_pos':line_data_pos,
        'data_neg':line_data_neg,
    }
    return response

In [40]:
query_keywords = ['黃珊珊']
cond='or'
weeks=12
cate='全部'
prepare_for_response(query_keywords, cond, cate, weeks)

{'sentiCount': {'pos': 76, 'neg': 34, 'obj': 23},
 'data_pos': [{'x': '2022-01-30', 'y': 0},
  {'x': '2022-02-06', 'y': 7},
  {'x': '2022-02-13', 'y': 5},
  {'x': '2022-02-20', 'y': 7},
  {'x': '2022-02-27', 'y': 1},
  {'x': '2022-03-06', 'y': 0},
  {'x': '2022-03-13', 'y': 2},
  {'x': '2022-03-20', 'y': 0},
  {'x': '2022-03-27', 'y': 12},
  {'x': '2022-04-03', 'y': 12},
  {'x': '2022-04-10', 'y': 5},
  {'x': '2022-04-17', 'y': 16},
  {'x': '2022-04-24', 'y': 9}],
 'data_neg': [{'x': '2022-01-30', 'y': 0},
  {'x': '2022-02-06', 'y': 2},
  {'x': '2022-02-13', 'y': 5},
  {'x': '2022-02-20', 'y': 3},
  {'x': '2022-02-27', 'y': 3},
  {'x': '2022-03-06', 'y': 0},
  {'x': '2022-03-13', 'y': 0},
  {'x': '2022-03-20', 'y': 2},
  {'x': '2022-03-27', 'y': 3},
  {'x': '2022-04-03', 'y': 3},
  {'x': '2022-04-10', 'y': 3},
  {'x': '2022-04-17', 'y': 5},
  {'x': '2022-04-24', 'y': 5}]}

# Step 5: Django views.py 請看示範專案APP 重複的模組要跟別人借用
    (1)app name: app_userkey_sentiment
    (2)namespace defined in urls.py: 
        app_name="namespace_userkey_sentiment"  
        or app_name="app_userkey_sentiment"
    (3) home.html


